In [6]:
!pip install pandas psycopg2 sqlalchemy


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from dotenv import load_dotenv
import os
import psycopg2

# Load environment variables from .env file
load_dotenv()

conn = psycopg2.connect(
    host=os.getenv("PG_HOST"),
    database=os.getenv("PG_DATABASE"),
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASSWORD"),
    sslmode=os.getenv("PG_SSLMODE", "require")  # Default to 'require'
)


In [7]:
import os
import json
import pandas as pd
import psycopg2


cursor = conn.cursor()

# Create history and collection table

In [10]:
cursor.execute("""
CREATE TABLE collect (
    id SERIAL PRIMARY KEY,
    username VARCHAR(255) NOT NULL,
    business_id VARCHAR(255) NOT NULL,
    collected_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

""")

In [ ]:
cursor.execute("""
CREATE TABLE click_history (
    id SERIAL PRIMARY KEY,            -- Auto-incrementing unique identifier for each record
    username VARCHAR(255) NOT NULL,  -- Username of the user (max 255 characters)
    recommendmodel VARCHAR(255),     -- Recommended model (optional, max 255 characters)
    click_store VARCHAR(255),        -- Store clicked (optional, max 255 characters)
    stay_time INTERVAL,              -- Stay time (using INTERVAL data type for time duration)
    collect BOOLEAN DEFAULT FALSE,   -- Collection status (1 or 0, stored as TRUE/FALSE)
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP -- Automatically record creation time
);
""")

In [14]:
conn.commit()

In [7]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS business (
    business_id VARCHAR PRIMARY KEY,
    name VARCHAR,
    latitude FLOAT,
    longitude FLOAT,
    stars FLOAT,
    review_count INTEGER,
    categories TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS review (
    review_id VARCHAR PRIMARY KEY,
    user_id VARCHAR,
    business_id VARCHAR,
    stars VARCHAR,
    text VARCHAR
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
    user_id VARCHAR PRIMARY KEY,
    name VARCHAR,
    review_count INTEGER,
    useful INTEGER,
    fans INTEGER
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS checkin (
    business_id VARCHAR,
    date TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS tip (
    user_id VARCHAR,
    business_id VARCHAR,
    text TEXT,
    date TIMESTAMP,
    compliment_count INTEGER
)
""")
conn.commit()

# My users

In [3]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS myusers (
    id SERIAL PRIMARY KEY,
    username VARCHAR(50) UNIQUE NOT NULL,
    password_hash TEXT NOT NULL
)
""")

cursor.execute("""
    ALTER TABLE myusers 
    ADD COLUMN IF NOT EXISTS cuisine_preference VARCHAR(255),
    ADD COLUMN IF NOT EXISTS taste_preference VARCHAR(255),
    ADD COLUMN IF NOT EXISTS special_requirements TEXT;
""")
conn.commit()

In [9]:
data_folder = './data/'

# Process Business

In [10]:
business_file = f'{data_folder}/yelp_academic_dataset_business.json'
business_data = []
with open(business_file, 'r', encoding='utf-8') as f:
    for line in f:
        business_data.append(json.loads(line))

In [11]:
business_df = pd.DataFrame(business_data)
business_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3..."
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."


In [12]:
business_philadelphia_df = business_df[business_df['city'] == 'Philadelphia']
business_philadelphia_df = business_philadelphia_df[['business_id', 'name', 'latitude', 'longitude', 'stars', 'review_count', 'categories']]
business_philadelphia_df

,business_id,name,latitude,longitude,stars,review_count,categories
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,39.955505,-75.155564,4.0,80,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
15,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,39.953949,-75.143226,4.0,245,"Sushi Bars, Restaurants, Japanese"
19,ROeacJQwBeh05Rqg7F6TCg,BAP,39.943223,-75.162568,4.5,205,"Korean, Restaurants"
28,QdN72BWoyFypdGJhhI5r7g,Bar One,39.939825,-75.157447,4.0,65,"Cocktail Bars, Bars, Italian, Nightlife, Resta..."
31,Mjboz24M9NlBeiOJKLEd_Q,DeSandro on Main,40.022466,-75.218314,3.0,41,"Pizza, Restaurants, Salad, Soup"
...,...,...,...,...,...,...,...
150320,wPAaAbFbq5wfLfW0gnYU7g,Intermix,39.950060,-75.170129,2.0,10,"Fashion, Shopping, Women's Clothing"
150326,JDM7kiCEmeBc_5W-z6eZZw,2601 Parkway Apartments,39.968806,-75.180608,3.0,6,"Real Estate, Apartments, Home Services"
150329,9U1Igcpe954LoWZRmNc-zg,Hand & Stone Massage And Facial Spa,39.932756,-75.144504,3.0,32,"Day Spas, Beauty & Spas, Skin Care, Massage"
150334,LJ4GjQ1HL6kqvIPpNUNNaQ,Shanti Yoga and Ayurveda,39.945966,-75.169666,4.5,39,"Health & Medical, Yoga, Shopping, Naturopathic..."


In [ ]:
for i, row in business_philadelphia_df.iterrows():
    cursor.execute("""
    INSERT INTO business (business_id, name, latitude, longitude, stars, review_count, categories)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (business_id) DO NOTHING
    """, (row['business_id'], row['name'], row['latitude'], row['longitude'], row['stars'], row['review_count'], row['categories']))
conn.commit()

In [7]:
# test

cursor.execute("SELECT * FROM business")
rows = cursor.fetchall()


print(len(rows))

14569


# Process Review

In [13]:
business_philadelphia_ids = set(business_philadelphia_df['business_id'].tolist())
len(business_philadelphia_ids)

14569

In [8]:
chunk_size = 10000  
review_file = f'{data_folder}/yelp_academic_dataset_review.json'

filtered_reviews = []


with open(review_file, 'r', encoding='utf-8') as f:
    chunk = []
    for line in f:
        review = json.loads(line)
        if review['business_id'] in business_philadelphia_ids:
            chunk.append(review)
        
        if len(chunk) >= chunk_size:
            chunk_df = pd.DataFrame(chunk)
            filtered_reviews.append(chunk_df)
            chunk = [] 

    if chunk:
        chunk_df = pd.DataFrame(chunk)
        filtered_reviews.append(chunk_df)


review_philadelphia_df = pd.concat(filtered_reviews, ignore_index=True)

In [ ]:
review_philadelphia_df.to_csv('review_philadelphia.csv', index=False)

In [2]:
import pandas as pd
review_philadelphia_df = pd.read_csv('review_philadelphia.csv')

In [ ]:
import concurrent.futures
import psycopg2
from datetime import datetime
conn = psycopg2.connect(
    host="cf980tnnkgv1bp.cluster-czrs8kj4isg7.us-east-1.rds.amazonaws.com",
    database="d3v92u2hcdeoqn",
    user="uabukrn1ju93p6",
    password="p8c12b2a0e1211fdcc0709d8b67b231bc8c492d12709db84c9b2e1fa4f374d7a6",
    sslmode='require' 
)

cursor = conn.cursor()

def insert_review(row):
    try:
        review_id = str(row.get('review_id', ''))
        user_id = str(row.get('user_id', ''))
        business_id = str(row.get('business_id', ''))
        stars = str(row.get('stars', '')) 
        text = str(row.get('text', '')).replace('\n', ' ').replace('\r', '') 
        

        # print(f"Executing query with values: {review_id}, {user_id}, {business_id}, {stars}, {text}")


        cursor.execute("""
        INSERT INTO review (review_id, user_id, business_id, stars, text)
        VALUES (%s, %s, %s, %s, %s)
        ON CONFLICT (review_id) DO NOTHING
        """, (review_id, user_id, business_id, stars, text))

        conn.commit()
    except Exception as e:
        print(f"Error inserting row: {e}")

def parallel_insert(df):
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        executor.map(insert_review, [row for _, row in df.iterrows()])


parallel_insert(review_philadelphia_df)


# Photo table

In [3]:
cursor.execute("""
CREATE TABLE photos (
    photo_id VARCHAR(255) PRIMARY KEY,      -- Unique identifier for each photo
    business_id VARCHAR(255),              -- Business ID associated with the photo
    caption TEXT,                           -- Caption of the photo
    label VARCHAR(50)                       -- Label describing the photo (e.g., food, drink, inside, outside)
);

""")

conn.commit()


In [4]:
import json
with open("./data/yelp_photos/photos_cleaned.json", "r") as file:
    for line in file:
        photo = json.loads(line)
        query = """
            INSERT INTO photos (photo_id, business_id, caption, label)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (photo_id) DO NOTHING
        """
        cursor.execute(query, (photo["photo_id"], photo["business_id"], photo["caption"], photo["label"]))


conn.commit()

In [7]:
cursor.close()
conn.close()


In [16]:
photos_dir = "./data/yelp_photos/photos" 
json_file_path = "./data/yelp_photos/photos.json" 

In [17]:
cleaned_json_path = "./data/yelp_photos/photos_cleaned.json"

with open(json_file_path, "r") as file, open(cleaned_json_path, "w") as cleaned_file:
    for line in file:
        photo = json.loads(line)
        if photo["business_id"] in business_philadelphia_ids:
            cleaned_file.write(json.dumps(photo) + "\n")


In [18]:
cleaned_json_path

'./data/yelp_photos/photos_cleaned.json'

In [19]:
import os
import json
import shutil


original_photos_dir = "./data/yelp_photos/photos" 
new_photos_dir = "./data/yelp_photos/photos_cleaned"  
json_file_path = "./data/yelp_photos/photos_cleaned.json" 

os.makedirs(new_photos_dir, exist_ok=True)

with open(json_file_path, "r") as file:
    cleaned_photos = [json.loads(line) for line in file]

for photo in cleaned_photos:
    photo_id = photo["photo_id"]
    original_photo_path = os.path.join(original_photos_dir, f"{photo_id}.jpg")
    new_photo_path = os.path.join(new_photos_dir, f"{photo_id}.jpg")

    if os.path.exists(original_photo_path):
        shutil.copy(original_photo_path, new_photo_path)
        print(f"Copied: {photo_id}.jpg")
    else:
        print(f"Photo not found: {photo_id}.jpg")


Copied: Le9rMdT8YFlvqr431LctIQ.jpg
Copied: zNzVcwnSJ4kvjFnANIsIRg.jpg
Copied: J1rqVl8pAoMJtPfGA2HV9w.jpg
Copied: 56xUu0i5oOBj9GdZqIg9_w.jpg
Copied: JyUo-KIUp3JJwHgS9FkLlA.jpg
Copied: GZpflvLA8AvQ6zi8aerdHg.jpg
Copied: Z0fXmxtTQ9PT2JFx8nnUOQ.jpg
Copied: rCHcbJ2NcwvsHzH1CFzyEw.jpg
Copied: vQ4f78kbIvE52xZBMKi65w.jpg
Copied: hKx0grJ77BUGqYzQ02VlrA.jpg
Copied: vDde56nmCntzYAMF7ZENVQ.jpg
Copied: kCxfhuZTQNRTM_cGhvdrWg.jpg
Copied: RsFf9wlW47mF6PmPYDuWXg.jpg
Copied: xYqXYXyN1riS74foNC0t0Q.jpg
Copied: uEFykMLeKCjz4y5sLBoG9w.jpg
Copied: cyzJVw9Xx5akGxmCqHlecw.jpg
Copied: yymIYHYGJo7O2ddBc6POzw.jpg
Copied: PJ38A9seHdxDit5bUImQgQ.jpg
Copied: ya7TOYAuJNEwv1n5_qfyBA.jpg
Copied: KJ5onXgXPn55XLlIQ7s6Yg.jpg
Copied: MdTTgu3f3rC7zyzZZ4mrfA.jpg
Copied: 2930C9zsK9UY429HCbEoBw.jpg
Copied: wbf2CGKMy-QQ8_ZHLCQivQ.jpg
Copied: W2g2-CzaNBH5WehtZ3MBJA.jpg
Copied: Og_ZiESCLTIXv0zrC6dhaA.jpg
Copied: Ovta76oHlqk1p5EE34K4RA.jpg
Copied: 2zzoTlZsLka7MokP77MdUg.jpg
Copied: n1YipiSexgt5b02xHT3yTQ.jpg
Copied: szJlMiDU72JG

In [2]:
! pip install pydrive

     ---------------------------------------- 0.0/987.4 kB ? eta -:--:--
     --------- ---------------------------- 256.0/987.4 kB 7.9 MB/s eta 0:00:01
     ------------------------------------- 987.4/987.4 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---- ----------------------------------- 1.5/12.6 MB 47.0 MB/s eta 0:00:01
   ---------- ----------------------------- 3.3/12.6 MB 42.3 MB/s eta 0:00:01
   ----------------- ---------------------- 5.4/12.6 MB 42.8 MB/s eta 0:00:01
   ---------------------- ----------------- 7.1/12.6 MB 41.5 MB/s eta 0:00:01
   ----------------------------- ---------- 9.3/12.6 MB 42.5 MB/s eta 0:00:01
   ------------------------------------ --- 11.3/12.6 MB 40.9 MB/s eta 0:00:01
   ---------------------------------------  12.6/12.6 MB 43.


[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
